In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/spotify_millsongdata.csv")

In [ ]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [ ]:
df.shape

(57650, 4)

In [ ]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [ ]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
df.head(10)

,artist,song,text
0,Wanda Jackson,Let's Say Goodbye Like We Said Hello,Let's say goodbye like we said hello in a frie...
1,Nina Simone,Do What You Gotta Do,Jimmy Webb \r\n \r\nDo what you gotta do \r...
2,Vertical Horizon,Echo,"[Chorus:] \r\nEcho, \r\nEcho-ooo \r\nWe com..."
3,Lil Wayne,Hot Revolver,"Young Mullah, baby \r\nHmm, Young Mullah, bab..."
4,Idina Menzel,Asleep On The Wind,Love is a glass of wine \r\nIt's balanced on ...
5,Garth Brooks,Fishin' In The Dark,Lazy yellow moon coming up tonight \r\nShinin...
6,George Michael,Cowboys And Angels,When your heart's in someone else's hands \r\...
7,Alice In Chains,Check My Brain,"So I found myself in the sun, oh yeah \r\nA h..."
8,Janis Joplin,Class A Jackass,Did you think I'd stand there \r\nWhile you k...
9,Styx,Krakatoa,"Written by John Curulewski, Paul Beaver, B. L...."


In [ ]:
df['text'][0]

"Let's say goodbye like we said hello in a friendly kind of way  \r\nThere's something wrong your love is gone I've no reason now to stay  \r\nWe live and love we meet and part and broken hearts must pay  \r\nLet's say goodbye like we said hello in a friendly kind of way  \r\nYour love sweetheart I won't forget you're always on my mind  \r\nYou're all the happiness I knew so thoughtful sweet and kind  \r\nFor old time sake I'm asking you to think of me some day  \r\nAnd let's say goodbye like we said hello in a friendly kind of way  \r\n[ guitar ]  \r\nI wish you happiness sweetheart in everything you do  \r\nMay good luck always come your way and every dream come true  \r\nI'll remember you and the love we knew our love of yesterday  \r\nSo let's say goodbye like we said hello in a friendly kind of way\r\n\r\n"

In [ ]:
# df = df.sample(5000)

In [ ]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [ ]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [ ]:
similarity[0]

array([1.        , 0.14288738, 0.0858961 , ..., 0.04672523, 0.05113238,
       0.00756131])

In [ ]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [ ]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [ ]:
recommendation('Asleep On The Wind')

['Keep Yourself Alive',
 "I'm Alive",
 'Stay Alive',
 'In Your Honor',
 'Say My Name',
 'Dead To The World',
 "I Don't Want To Live Forever",
 'Wind Me Up',
 'Staying Alive',
 "Don't Let It End (Reprise)",
 'Love',
 'You Can Touch',
 'Idioteque',
 'The Messenger',
 "She's Alive",
 'Follow The Wind',
 'High Wire - The Aerialist',
 'Long Way Down',
 'Alive',
 'Good Girl']

In [ ]:
import pickle


In [ ]:
pip install pandas==1.3.5


In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))